In [1]:
print(2)

2


In [5]:
import torch
from transformer import Transformer

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

trg_vocab = list(set(' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890.,#!@"$;:^|\'-=+_—?<>'))
src_vocab = list(set(' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890.,#!@"$;:^|\'-=+_—?<>'))

trg_vocab.append(START_TOKEN)
trg_vocab.append(END_TOKEN)
trg_vocab.append(PADDING_TOKEN)

src_vocab.append(START_TOKEN)
src_vocab.append(END_TOKEN)
src_vocab.append(PADDING_TOKEN)

index_to_trg = {k:v for k,v in enumerate(trg_vocab)}
trg_to_index = {v:k for k,v in enumerate(trg_vocab)}
index_to_src = {k:v for k,v in enumerate(src_vocab)}
src_to_index = {v:k for k,v in enumerate(src_vocab)}

d_model = 512
batch_size = 50
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.2
num_layers = 1
max_sequence_length = 200
trg_vocab_size = len(trg_vocab)

ru_eng_translator = Transformer(d_model, 
                            ffn_hidden,
                            num_heads, 
                            drop_prob, 
                            num_layers, 
                            max_sequence_length,
                            trg_vocab_size,
                            src_to_index,
                            trg_to_index,
                            index_to_trg,
                            START_TOKEN, 
                            END_TOKEN, 
                            PADDING_TOKEN)

eng_ru_translator = Transformer(d_model, 
                            ffn_hidden,
                            num_heads, 
                            drop_prob, 
                            num_layers, 
                            max_sequence_length,
                            trg_vocab_size,
                            src_to_index,
                            trg_to_index,
                            index_to_trg,
                            START_TOKEN, 
                            END_TOKEN, 
                            PADDING_TOKEN)

ru_eng_translator.load_state_dict(torch.load('models/ru_eng_trans.pth'))
eng_ru_translator.load_state_dict(torch.load('models/eng_ru_trans.pth'))

<All keys matched successfully>

In [6]:
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device = get_device()

In [7]:
ru_eng_translator.to(device)
eng_ru_translator.to(device)

ru_eng_translator.eval()
eng_ru_translator.eval()

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(152, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (ln): LayerNorm((512,), eps=512, elementwise_affine=True)
        )
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (ffn): FeedFoward(
          (net): Sequential(
            (0): Linear(in_features=512, out_features=2048, bias=True)
            (1): ReLU()
            (2): Linear(in_features=2048, out_features=512, bias=True)
            (3): Dropout(p=0.2, inplace=False)
          )
        )
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout2): Dropout(p=0.2, inplace=F

In [11]:
print(ru_eng_translator.state_dict())

OrderedDict([('encoder.sentence_embedding.embedding.weight', tensor([[ 0.0044,  0.0764,  0.0221,  ..., -0.0933,  0.0078,  0.0835],
        [-0.0497, -0.0095,  0.0531,  ..., -0.0244, -0.0254, -0.0178],
        [ 0.1046,  0.0684, -0.0545,  ...,  0.0625, -0.0367,  0.0217],
        ...,
        [-0.0720,  0.0223,  0.0409,  ...,  0.0092,  0.0814,  0.0739],
        [-0.0713,  0.0091,  0.0162,  ..., -0.0404,  0.0625, -0.0015],
        [-0.0011,  0.0746, -0.1186,  ..., -0.0788,  0.0516, -0.0465]],
       device='cuda:0')), ('encoder.layers.0.attention.qkv_layer.weight', tensor([[-0.0593,  0.0162,  0.0068,  ..., -0.0028, -0.0376, -0.0105],
        [ 0.0417, -0.0444,  0.0554,  ..., -0.0065,  0.0941, -0.0387],
        [-0.0395, -0.0588,  0.0103,  ...,  0.0236, -0.0986,  0.0004],
        ...,
        [ 0.1378, -0.0574, -0.1130,  ...,  0.0363,  0.1418, -0.0203],
        [-0.0122, -0.0157, -0.0370,  ..., -0.0508,  0.1853, -0.0450],
        [-0.1752,  0.0730, -0.0588,  ..., -0.0375, -0.2896,  0.0143]

In [9]:
ru_eng_translator.translate("Привет, как дела?")

'вzl,?Bрп,mKlB ЛKBд,#NK# у,#'

In [28]:
eng_ru_translator.translate("Hi, how are you?")

'ЙR<ULТJR<ЫbVЫbНVKТLТnШR'

In [2]:
import torch

In [3]:
model1 = torch.load("models/eng_ru_trans.pt")
model2 = torch.load("models/ru_eng_trans.pt")

<class 'collections.OrderedDict'>


In [16]:
from transformer import *

In [17]:
def translate(model, src_sentence):
        
        model.eval()
        max_sequence_length = model.max_sequence_length
        device = get_device()
        src_sentence = (src_sentence,)
        trg_sentence = ("",)
        
        for word_counter in range(max_sequence_length):

            encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask=create_masks(src_sentence, trg_sentence)
            predictions = model(src_sentence,
                                    trg_sentence,
                                    encoder_self_attention_mask.to(device), 
                                    decoder_self_attention_mask.to(device), 
                                    decoder_cross_attention_mask.to(device),
                                    enc_start_token=False,
                                    enc_end_token=False,
                                    dec_start_token=True,
                                    dec_end_token=False)
            
            next_token_prob_distribution = predictions[0][word_counter]
            next_token_index = torch.argmax(next_token_prob_distribution).item()
            next_token = model.index_to_trg[next_token_index]
            
            if next_token == model.END_TOKEN:
                break

            trg_sentence = (trg_sentence[0] + next_token, )
        return trg_sentence[0]

In [18]:
translate(model1, "wad")

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [4]:
import torch
from transformer import Transformer

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

trg_vocab = sorted(list(set(' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890.,#!@"$;:^|\'-=+_—?<>')))
src_vocab = sorted(list(set(' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890.,#!@"$;:^|\'-=+_—?<>')))

trg_vocab.append(START_TOKEN)
trg_vocab.append(END_TOKEN)
trg_vocab.append(PADDING_TOKEN)

src_vocab.append(START_TOKEN)
src_vocab.append(END_TOKEN)
src_vocab.append(PADDING_TOKEN)

index_to_trg = {k:v for k,v in enumerate(trg_vocab)}
trg_to_index = {v:k for k,v in enumerate(trg_vocab)}
index_to_src = {k:v for k,v in enumerate(src_vocab)}
src_to_index = {v:k for k,v in enumerate(src_vocab)}

d_model = 512
batch_size = 50
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.2
num_layers = 1
max_sequence_length = 200
trg_vocab_size = len(trg_vocab)

test_model = Transformer(d_model, 
                            ffn_hidden,
                            num_heads, 
                            drop_prob, 
                            num_layers, 
                            max_sequence_length,
                            trg_vocab_size,
                            src_to_index,
                            trg_to_index,
                            index_to_trg,
                            START_TOKEN, 
                            END_TOKEN, 
                            PADDING_TOKEN)

test_model.load_state_dict(torch.load("models/test_model.pth"))

def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device = get_device()

test_model.to(device)

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(152, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (ln): LayerNorm((512,), eps=512, elementwise_affine=True)
        )
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (ffn): FeedFoward(
          (net): Sequential(
            (0): Linear(in_features=512, out_features=2048, bias=True)
            (1): ReLU()
            (2): Linear(in_features=2048, out_features=512, bias=True)
            (3): Dropout(p=0.2, inplace=False)
          )
        )
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout2): Dropout(p=0.2, inplace=F

In [5]:
print(test_model.max_sequence_length)
print(test_model.trg_vocab_size)

print(test_model.trg_to_index)
print(test_model.index_to_trg)

print(test_model.PADDING_TOKEN)
print(test_model.START_TOKEN)
print(test_model.END_TOKEN)

print(test_model.device)

200
152
{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, "'": 5, '+': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '<': 22, '=': 23, '>': 24, '?': 25, '@': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 37, 'L': 38, 'M': 39, 'N': 40, 'O': 41, 'P': 42, 'Q': 43, 'R': 44, 'S': 45, 'T': 46, 'U': 47, 'V': 48, 'W': 49, 'X': 50, 'Y': 51, 'Z': 52, '^': 53, '_': 54, 'a': 55, 'b': 56, 'c': 57, 'd': 58, 'e': 59, 'f': 60, 'g': 61, 'h': 62, 'i': 63, 'j': 64, 'k': 65, 'l': 66, 'm': 67, 'n': 68, 'o': 69, 'p': 70, 'q': 71, 'r': 72, 's': 73, 't': 74, 'u': 75, 'v': 76, 'w': 77, 'x': 78, 'y': 79, 'z': 80, '|': 81, 'Ё': 82, 'А': 83, 'Б': 84, 'В': 85, 'Г': 86, 'Д': 87, 'Е': 88, 'Ж': 89, 'З': 90, 'И': 91, 'Й': 92, 'К': 93, 'Л': 94, 'М': 95, 'Н': 96, 'О': 97, 'П': 98, 'Р': 99, 'С': 100, 'Т': 101, 'У': 102, 'Ф': 103, 'Х': 104, 'Ц': 105, 'Ч': 106, 'Ш': 107, 'Щ': 108, 'Ъ': 109, '

In [7]:
test_model.translate("Привет, как дела?")

'An an titin tin'

In [13]:
print(test_model.state_dict())

OrderedDict([('encoder.sentence_embedding.embedding.weight', tensor([[ 2.4216e-03, -9.1297e-02, -1.7591e-02,  ..., -6.4251e-02,
          5.8866e-02,  6.6647e-02],
        [ 2.6140e-02,  5.9015e-05,  5.1235e-02,  ...,  4.4288e-02,
         -5.7951e-02,  1.3084e-02],
        [ 6.0429e-02, -8.9595e-02,  1.0528e-01,  ..., -4.2966e-02,
         -7.7715e-02,  8.0295e-02],
        ...,
        [-2.1345e-02, -5.2924e-02, -8.6360e-02,  ..., -3.1504e-02,
         -5.6335e-02, -8.5197e-02],
        [ 7.2006e-02,  3.9475e-02,  3.8702e-02,  ...,  5.7872e-02,
          7.1164e-03, -5.6769e-02],
        [ 4.6184e-02, -7.9580e-02,  8.9015e-02,  ..., -8.1804e-02,
         -6.3777e-02,  9.3205e-02]], device='cuda:0')), ('encoder.layers.0.attention.qkv_layer.weight', tensor([[-0.0416, -0.0023, -0.0396,  ...,  0.0519, -0.0513,  0.0012],
        [-0.0057,  0.0296,  0.0244,  ..., -0.0438, -0.0510,  0.0233],
        [ 0.0432,  0.0349,  0.0036,  ...,  0.0222, -0.0048,  0.0320],
        ...,
        [ 0.0363,

In [16]:
print(test_model.max_sequence_length)
print(test_model.trg_vocab_size)

print(test_model.trg_to_index)
print(test_model.index_to_trg)

print(test_model.PADDING_TOKEN)
print(test_model.START_TOKEN)
print(test_model.END_TOKEN)

print(test_model.device)

200
152
{'O': 0, 'H': 1, 'o': 2, '@': 3, "'": 4, 'h': 5, 'щ': 6, 'х': 7, 'Ё': 8, 'К': 9, 'И': 10, 'a': 11, '8': 12, '4': 13, 'z': 14, 'p': 15, 'r': 16, 't': 17, 'г': 18, 'м': 19, 'v': 20, 'm': 21, 'П': 22, 'В': 23, 'Г': 24, 'С': 25, 'о': 26, '!': 27, 'i': 28, 'R': 29, 'ё': 30, 'п': 31, 'Ы': 32, 'J': 33, '?': 34, 'Щ': 35, 'у': 36, 'Z': 37, 'я': 38, 'ь': 39, '.': 40, 'ф': 41, 'Э': 42, 'М': 43, 'с': 44, 'ю': 45, '#': 46, 'З': 47, 'x': 48, 'N': 49, 'l': 50, 'd': 51, 'Р': 52, 'n': 53, 'T': 54, 'л': 55, '_': 56, '9': 57, 'B': 58, 'Л': 59, 'V': 60, 'Ч': 61, 'D': 62, 'з': 63, 'Ю': 64, 'ш': 65, 'д': 66, 'Ь': 67, '=': 68, 'Б': 69, 'Т': 70, 'У': 71, 'ц': 72, ':': 73, 'Ц': 74, 'y': 75, 'u': 76, '|': 77, 'f': 78, 'P': 79, 'g': 80, 'c': 81, 'Ж': 82, ',': 83, 'н': 84, 'э': 85, 'E': 86, '2': 87, 'Ъ': 88, 'Ш': 89, '5': 90, '6': 91, 'U': 92, 'к': 93, 'Н': 94, 'ъ': 95, '1': 96, 'Ф': 97, 'k': 98, 'Й': 99, '-': 100, ';': 101, 'ч': 102, 'C': 103, 'G': 104, '>': 105, 'Q': 106, 'L': 107, 'e': 108, 'Х': 109, '

In [21]:
vocab = sorted(list(set(' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZабвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890.,#!@"$;:^|\'-=+_—?<>')))

vocab.append(START_TOKEN)
vocab.append(END_TOKEN)
vocab.append(PADDING_TOKEN)

index_to_voc = {k:v for k,v in enumerate(vocab)}
voc_to_index = {v:k for k,v in enumerate(vocab)}

print(index_to_voc)
print(voc_to_index)

{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: "'", 6: '+', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21: ';', 22: '<', 23: '=', 24: '>', 25: '?', 26: '@', 27: 'A', 28: 'B', 29: 'C', 30: 'D', 31: 'E', 32: 'F', 33: 'G', 34: 'H', 35: 'I', 36: 'J', 37: 'K', 38: 'L', 39: 'M', 40: 'N', 41: 'O', 42: 'P', 43: 'Q', 44: 'R', 45: 'S', 46: 'T', 47: 'U', 48: 'V', 49: 'W', 50: 'X', 51: 'Y', 52: 'Z', 53: '^', 54: '_', 55: 'a', 56: 'b', 57: 'c', 58: 'd', 59: 'e', 60: 'f', 61: 'g', 62: 'h', 63: 'i', 64: 'j', 65: 'k', 66: 'l', 67: 'm', 68: 'n', 69: 'o', 70: 'p', 71: 'q', 72: 'r', 73: 's', 74: 't', 75: 'u', 76: 'v', 77: 'w', 78: 'x', 79: 'y', 80: 'z', 81: '|', 82: 'Ё', 83: 'А', 84: 'Б', 85: 'В', 86: 'Г', 87: 'Д', 88: 'Е', 89: 'Ж', 90: 'З', 91: 'И', 92: 'Й', 93: 'К', 94: 'Л', 95: 'М', 96: 'Н', 97: 'О', 98: 'П', 99: 'Р', 100: 'С', 101: 'Т', 102: 'У', 103: 'Ф', 104: 'Х', 105: 'Ц', 106: 'Ч', 107: 'Ш', 108: 'Щ', 109: 'Ъ', 110: 'Ы',

In [22]:
print(sorted(vocab))

[' ', '!', '"', '#', '$', "'", '+', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '<END>', '<PADDING>', '<START>', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', 'Ё', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', '—']


In [23]:
vocab

[' ',
 '!',
 '"',
 '#',
 '$',
 "'",
 '+',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '^',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 'Ё',
 'А',
 'Б',
 'В',
 'Г',
 'Д',
 'Е',
 'Ж',
 'З',
 'И',
 'Й',
 'К',
 'Л',
 'М',
 'Н',
 'О',
 'П',
 'Р',
 'С',
 'Т',
 'У',
 'Ф',
 'Х',
 'Ц',
 'Ч',
 'Ш',
 'Щ',
 'Ъ',
 'Ы',
 'Ь',
 'Э',
 'Ю',
 'Я',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё',
 '—',
 '<START>',
 '<END>',
 '<PADDING>']